In [2]:
# feature extraction consists of taking the convolutional base of a pre-existing network and stick a new densely connected layer

In [3]:
from keras.applications import VGG16 # TODO: try also Xception, Inception V3, ResNet50, VGG16, VGG19, MobileNet and others domain-specific nets
from keras.applications import Xception
from keras.applications import ResNet50
from keras.applications import EfficientNetB2
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
import pandas as pd

2022-12-02 14:36:37.245301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["dlopen(/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so, 0x0006): symbol not found in flat namespace '__ZN3tsl2io7DirnameENSt3__117basic_string_viewIcNS1_11char_traitsIcEEEE'"]
  warnings.warn(f"unable to load libtensorflow_io_pl

In [4]:
# hyperparameters to be set
input_height = 1920
input_width = 1080
# others

In [5]:
conv_base = VGG16(weights='imagenet', # weight checkpoint from which we initialize the model
                  include_top=False, # to decide if need to include the densely connectedd layer
                  input_shape=(input_width, input_height, 3)) #TODO: check order

In [6]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1080, 1920, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 1080, 1920, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 1080, 1920, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 540, 960, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 540, 960, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 540, 960, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 270, 480, 128)     0     

# 1. First approach: run convolutional base over inputs, save in array and then give it to a standalone densely connected layer (approach cannot use data augmentation)

In [15]:
images_path = os.path.join("..","workspace","images")
train_dir = os.path.join(images_path, "train")
validation_dir = os.path.join(images_path, "eval")
test_dir = os.path.join(images_path, "test")
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [12]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512)) # (4,4,512) is the VGG16 final feature map shape
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')
    i=0
    for inputs_batch, labels_batch in generator:
        # we extract features by calling the predict method of the conv_base model
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [ ]:
count = 0
for filename in os.listdir(train_dir):
    count += 1
train_features, train_labels = extract_features(train_dir, count)

Found 153 validated image filenames.


In [ ]:
count = 0
for filename in os.listdir(validation_dir):
    count += 1
validation_features, validation_labels = extract_features(validation_dir, count)

In [ ]:
count = 0
for filename in os.listdir(test_dir):
    count += 1
test_features, test_labels = extract_features(test_dir, count)

In [ ]:
# best loss function for multi-class classification, measures the distance between two probability distributions
# the probability distribution of the output of the network and the true distribution of the labels
loss_function='categorical_crossentropy'

metrics=['accuracy']
optimizer='rmsprop'
optimizer_learning_rate=1e-4
epochs=5
batch_size=20
num_classes = 8

if optimizer == 'rmsprop':
    optimizer=optimizers.RMSprop(learning_rate=optimizer_learning_rate)

In [ ]:
# densely connected classifier on top
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=metrics)
history = model.fit(train_features, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_features, validation_labels))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation ACC')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# 2. Approach that extend the model and add a dense layer on top, running the whole thing end to end on the input data (support data augmentation)

In [17]:
images_path = os.path.join("..","workspace","images")
train_dir = os.path.join(images_path, "train")
validation_dir = os.path.join(images_path, "eval")
test_dir = os.path.join(images_path, "test")
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [7]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 33, 60, 512)       14714688  
                                                                 
 flatten (Flatten)           (None, 1013760)           0         
                                                                 
 dense (Dense)               (None, 256)               259522816 
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 274,237,761
Trainable params: 274,237,761
Non-trainable params: 0
_________________________________________________________________


In [12]:
print('This is the number of trainable weights '
         'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False # freeze conv_base otherwise representation previously learned got updated
# only the weights of the densely connected layer will be trained

This is the number of trainable weights before freezing the conv base: 30


In [13]:
print('This is the number of trainable weights '
          'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


In [ ]:
# best loss function for multi-class classification, measures the distance between two probability distributions
# the probability distribution of the output of the network and the true distribution of the labels
loss_function='categorical_crossentropy'

metrics=['accuracy']
optimizer='rmsprop'
optimizer_learning_rate=1e-4
epochs=5
batch_size=20
num_classes = 8

if optimizer == 'rmsprop':
    optimizer=optimizers.RMSprop(learning_rate=optimizer_learning_rate)

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest') # set right parameters

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
df_eval = pd.read_csv(os.path.join(validation_dir, "df_eval.csv"))
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=metrics)

history = model.fit_generator(
      train_generator,
      steps_per_epoch=10,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=5)

Found 153 validated image filenames.
Found 24 validated image filenames.


/Users/francoterranova/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_50095/3850630360.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation MAE')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

# if performances are much wors than validation ones, during hyperparameter optimization (when done) the process has overfitted the validdation set, if so go to a more clear protocol such as Kfold CV
test_loss, test_acc = model.evaluate_generator(test_generator, steps=2)
print('test acc:', test_acc)
print('test loss:', test_loss)